In [ ]:
from google.genai import types # type: ignore
from google import genai
import pandas as pd
import sys
import os
notebook_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
from main.GeminiClient import GeminiClient
import time
from main.MariaDBClient import MariaDBClient
from main.MarketScraper import MarketScraper
import pandas as pd
import asyncio

In [2]:
# https://www.listendata.com/2024/05/how-to-use-gemini-in-python.html
# Create API KEY: https://aistudio.google.com/app/u/0/apikey?pli=1

In [3]:
mariadbclient = MariaDBClient()
supermercados = mariadbclient.get_active_tables()
market_scraper = MarketScraper()
total_items = []
total_stats = []
for supermercado in supermercados:
    data = market_scraper.fetch_data(supermercado)
    items = market_scraper.get_items(data)
    print(supermercado, " : ", len(items))
    total_items.append(items)
    stas = market_scraper.get_stats(data)
    total_stats.append(stas)

total_items_df = pd.concat(total_items, ignore_index=True)
total_stats_df = pd.concat(total_stats, ignore_index=True)


atomo  :  365
blowmax  :  211
modomarket  :  405
segal  :  166


In [4]:
print(stas['elapsed_time_seconds'].iloc[0])


15.016275


In [5]:
new_items_df = total_items_df

In [6]:
old_items_df = mariadbclient.get_products()

In [7]:
new_only_df = new_items_df[~new_items_df['id_producto'].isin(old_items_df['id_producto'])] # type: ignore
common_df = new_items_df[new_items_df['id_producto'].isin(old_items_df['id_producto'])]
old_only_df = old_items_df[~old_items_df['id_producto'].isin(new_items_df['id_producto'])]
print(f"Nuevos productos: {new_only_df.shape[0]}")
print(f"Productos ya existentes: {common_df.shape[0]}")
print(f"Productos ya no existentes: {old_only_df.shape[0]}")

Nuevos productos: 815
Productos ya existentes: 332
Productos ya no existentes: 0


In [8]:
old_only_df

,__ingestion_timestamp,nombre_crudo,id_producto,categoria,nombre,marca,descripcion,volumen,precio,id_supermercado,id_grupo,id_subgrupo,url


In [9]:
mariadbclient.delete_products(old_only_df)
print("Se eliminaron ",len(old_only_df), " productos")
mariadbclient.update_products(common_df)
print("Se actualizaron ",len(common_df), " productos")

Se eliminaron  0  productos
Se actualizaron  332  productos


In [10]:
old_productos_to_update_subgroups_df = mariadbclient.get_products_by_groups(new_only_df)
new_productos_to_update_subgroups_df = new_only_df[['id_producto', 'categoria', 'marca', 'descripcion', 'volumen', 'id_supermercado']]
new_productos_to_update_subgroups_df = new_productos_to_update_subgroups_df.rename(columns={'id_supermercado': 'supermercado'})
new_productos_to_update_subgroups_df['supermercado'] = new_productos_to_update_subgroups_df['supermercado'].map({
    1:"Atomo",
    2:"Blowmax",
    3:"ModoMarket",
    4:"Segal",
    5:"Supera"
})
all_productos_to_upsert_subgroups_df = pd.concat([new_productos_to_update_subgroups_df, old_productos_to_update_subgroups_df], ignore_index=True)
all_productos_to_upsert_subgroups_df['producto_id'] = all_productos_to_upsert_subgroups_df['id_producto'].astype(str) + " - " + all_productos_to_upsert_subgroups_df['categoria'] + " - " + all_productos_to_upsert_subgroups_df['marca'] + " - " + all_productos_to_upsert_subgroups_df['descripcion'] + " - " + all_productos_to_upsert_subgroups_df['volumen'] + " - " + all_productos_to_upsert_subgroups_df['supermercado']


In [11]:
print("Hay ", len(old_productos_to_update_subgroups_df), " productos que comparten subgrupo con los productos nuevos")
print("Hay ", len(new_productos_to_update_subgroups_df), " productos nuevos que hay que asignar subgrupo")
print("Hay ", len(all_productos_to_upsert_subgroups_df), " productos que hay que actualizar su subgrupo")

Hay  238  productos que comparten subgrupo con los productos nuevos
Hay  815  productos nuevos que hay que asignar subgrupo
Hay  1053  productos que hay que actualizar su subgrupo


In [12]:
grouped = all_productos_to_upsert_subgroups_df.groupby(['categoria', 'marca', 'volumen'])
listas_productos = [group['producto_id'].tolist() for _, group in grouped]

geminiclient = GeminiClient()
geminiclient.api_keys

total_elementos = sum(len(sublista) for sublista in listas_productos)
all_responses = []

indice = 0
for lista_productos in listas_productos:
    print("clasificando: ", str(lista_productos))
    
    with open('agrupacion2.txt', 'a') as file: #!

        prompt = listas_productos[indice]
        try:
            if len(prompt) > 1:
                respuesta = geminiclient.ask(prompt)

                for lista in respuesta:
                    file.write(str(lista) + '\n') #!
                file.write('----'*10 + '\n') #!

                time.sleep(4.1)
            else:
                respuesta = [prompt]
            all_responses.append(respuesta)
        except Exception as e:
            print(f"Error al procesar el prompt: {prompt}")
            print(f"Error: {e}")
    indice += 1

clasificando:  ['807464942 - Arroces - ALA - LARGO FINO - 1000 - Atomo', '3090295548 - Arroces - ALA - PARBOIL - 1000 - Atomo', '2772439016 - Arroces - ALA - INTEGRAL - 1000 - Atomo']
clasificando:  ['2670982269 - Arroces - ALA - MOLINO CARNAROLI ESTUCHE - 500 - Atomo']
clasificando:  ['3449080988 - Arroces - APOSTOLES - LARGO FINO 00000 - 1000 - Atomo']
clasificando:  ['918436064 - Arroces - BARBARA - LARGO FINO - 1000 - Blowmax']
clasificando:  ['3901969118 - Arroces - BARBARA - LARGO FINO - 500 - Blowmax']
clasificando:  ['1319222139 - Arroces - DON MARCOS - LARGO FINO - 1000 - Segal', '2550908818 - Arroces - DON MARCOS - PARBOIL - 1000 - Segal']
clasificando:  ['3747584072 - Arroces - DOS HERMANOS - LARGO FINO - 1000 - ModoMarket', '2906657493 - Arroces - DOS HERMANOS - PARBOIL - 1000 - ModoMarket']
clasificando:  ['3936387000 - Arroces - EL GRANDE -  - 1000 - Blowmax']
clasificando:  ['233023545 - Arroces - EL GRANDE -  - 500 - Blowmax']
clasificando:  ['3616581821 - Arroces - GAL

16m 39s

In [13]:
print("Gemini tiene que agrupar ",total_elementos, " productos")
total_agrupados = sum(len(subsublist) for sublist in all_responses for subsublist in sublist)
print("Gemini agrupo ",total_agrupados, " productos")


Gemini tiene que agrupar  1053  productos
Gemini agrupo  1048  productos


In [14]:
all_responses

[[['807464942 - Arroces - ALA - LARGO FINO - 1000 - Atomo'],
  ['3090295548 - Arroces - ALA - PARBOIL - 1000 - Atomo'],
  ['2772439016 - Arroces - ALA - INTEGRAL - 1000 - Atomo']],
 [['2670982269 - Arroces - ALA - MOLINO CARNAROLI ESTUCHE - 500 - Atomo']],
 [['3449080988 - Arroces - APOSTOLES - LARGO FINO 00000 - 1000 - Atomo']],
 [['918436064 - Arroces - BARBARA - LARGO FINO - 1000 - Blowmax']],
 [['3901969118 - Arroces - BARBARA - LARGO FINO - 500 - Blowmax']],
 [['1319222139 - Arroces - DON MARCOS - LARGO FINO - 1000 - Segal'],
  ['2550908818 - Arroces - DON MARCOS - PARBOIL - 1000 - Segal']],
 [['3747584072 - Arroces - DOS HERMANOS - LARGO FINO - 1000 - ModoMarket'],
  ['2906657493 - Arroces - DOS HERMANOS - PARBOIL - 1000 - ModoMarket']],
 [['3936387000 - Arroces - EL GRANDE -  - 1000 - Blowmax']],
 [['233023545 - Arroces - EL GRANDE -  - 500 - Blowmax']],
 [['3616581821 - Arroces - GALLO - LARGO FINO SELEC - 1000 - Atomo'],
  ['678088978 - Arroces - GALLO - LARGO FINO LARGO FIN -

In [15]:
#TODO
#TODO EN el main si len(all_response) return, no hace falta seguir para abajo

In [16]:
olds_ids = set(old_productos_to_update_subgroups_df['id_producto'])
news_ids = set(new_items_df['id_producto'])

# Preparamos las estructuras de resultados
updates = []
descartados = []

# Creamos un diccionario para mapear rápidamente los IDs a sus índices en new_items_df
new_items_index = {id_producto: idx for idx, id_producto in enumerate(new_items_df['id_producto'])}

# Procesamos todas las respuestas
for respuesta in all_responses:
    for j, subgrupo in enumerate(respuesta):
        ids_productos = [item.partition(' - ')[0].strip() for item in subgrupo]  # .strip() para eliminar espacios
        
        for id_producto_str in ids_productos:
                id_producto = int(id_producto_str)
                
                if id_producto in olds_ids:
                    updates.append({ 
                        "id_producto": id_producto,
                        "id_subgrupo": j
                    })
                elif id_producto in news_ids:
                    idx = new_items_index[id_producto]
                    new_only_df.at[idx, 'id_subgrupo'] = j
                else:
                    descartados.append(id_producto_str)


In [17]:
new_only_df

,__ingestion_timestamp,nombre_crudo,id_producto,categoria,nombre,marca,descripcion,volumen,precio,id_supermercado,id_grupo,id_subgrupo,url
1,2025-05-03 11:45:07.738515-03:00,PAN REBANADO BIMBO BLANCO 400 GRS,486306186,Panes,BIMBO REBANADO BLANCO 400,BIMBO,REBANADO BLANCO,400,4390.0,1,4120442803,0.0,https://atomoconviene.com/atomo-ecommerce/pan-...
2,2025-05-03 11:45:07.738515-03:00,PAN REBANADO LACTAL GRANDE 460 GRS,2828291709,Panes,LACTAL REBANADO GRANDE 460,LACTAL,REBANADO GRANDE,460,3299.0,1,1887811491,0.0,https://atomoconviene.com/atomo-ecommerce/pan-...
3,2025-05-03 11:45:07.738515-03:00,PAN REBANADO LACTAL CHICO 315 GRS,3915559259,Panes,LACTAL REBANADO CHICO 315,LACTAL,REBANADO CHICO,315,2350.0,1,1013619072,0.0,https://atomoconviene.com/atomo-ecommerce/pan-...
4,2025-05-03 11:45:07.738515-03:00,PAN DE SALVADO LACTAL CHICO 330 GRS,2612372097,Panes,LACTAL DE SALVADO CHICO 330,LACTAL,DE SALVADO CHICO,330,2630.0,1,781113677,0.0,https://atomoconviene.com/atomo-ecommerce/inte...
5,2025-05-03 11:45:07.738515-03:00,PANADERIA MILLAN PAN REBANADO CON SEMILLA ARTE...,3144192866,Panes,MILLAN REBANADO CON SEMILLA ARTESANO 400,MILLAN,REBANADO CON SEMILLA ARTESANO,400,2470.0,1,2107761313,0.0,https://atomoconviene.com/atomo-ecommerce/pana...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1142,2025-05-03 11:48:50.693948-03:00,TRESEMMÉ SHAMPOO 400ml BAJO POO NUTRICIÓN,1818883456,Shampoos,TRESEMME BAJO POO NUTRICION 400,TRESEMME,BAJO POO NUTRICION,400,4569.0,4,2472537432,0.0,https://www.casa-segal.com/producto/tresemme-s...
1143,2025-05-03 11:48:50.693948-03:00,TRESEMMÉ SHAMPOO 400ml DETOX CAPILAR,911102848,Shampoos,TRESEMME DETOX CAPILAR 400,TRESEMME,DETOX CAPILAR,400,4569.0,4,2472537432,1.0,https://www.casa-segal.com/producto/tresemme-s...
1144,2025-05-03 11:48:50.693948-03:00,TRESEMMÉ SHAMPOO 400ml HIDRATACIÓN PROFUNDA,506018233,Shampoos,TRESEMME HIDRATACION PROFUNDA 400,TRESEMME,HIDRATACION PROFUNDA,400,4569.0,4,2472537432,2.0,https://www.casa-segal.com/producto/tresemme-s...
1145,2025-05-03 11:48:50.693948-03:00,TRESEMMÉ SHAMPOO 400ml REGENERACIÓN TRESPLEX,3715602674,Shampoos,TRESEMME REGENERACION TRESPLEX 400,TRESEMME,REGENERACION TRESPLEX,400,4569.0,4,2472537432,3.0,https://www.casa-segal.com/producto/tresemme-s...


In [18]:
updates

[{'id_producto': 3090295548, 'id_subgrupo': 1},
 {'id_producto': 2772439016, 'id_subgrupo': 2},
 {'id_producto': 1576117822, 'id_subgrupo': 7},
 {'id_producto': 85589716, 'id_subgrupo': 7},
 {'id_producto': 1856604834, 'id_subgrupo': 9},
 {'id_producto': 4244691140, 'id_subgrupo': 0},
 {'id_producto': 3697813574, 'id_subgrupo': 0},
 {'id_producto': 1684680083, 'id_subgrupo': 1},
 {'id_producto': 2219397944, 'id_subgrupo': 2},
 {'id_producto': 2804438064, 'id_subgrupo': 6},
 {'id_producto': 825363576, 'id_subgrupo': 1},
 {'id_producto': 57310437, 'id_subgrupo': 1},
 {'id_producto': 2431942202, 'id_subgrupo': 1},
 {'id_producto': 3257464427, 'id_subgrupo': 4},
 {'id_producto': 375723713, 'id_subgrupo': 5},
 {'id_producto': 957592185, 'id_subgrupo': 6},
 {'id_producto': 500609407, 'id_subgrupo': 7},
 {'id_producto': 481706055, 'id_subgrupo': 8},
 {'id_producto': 132450231, 'id_subgrupo': 9},
 {'id_producto': 813721280, 'id_subgrupo': 0},
 {'id_producto': 1469293990, 'id_subgrupo': 0},
 {'

In [19]:
descartados

[]

In [20]:
mariadbclient.insert_into_productos(new_only_df)
mariadbclient.update_product_subgroups(updates)